In [1]:
import pandas as pd 
import numpy as np

In [2]:
df = pd.read_csv("../csv/AAPL.csv", index_col="Date")
df.head(2)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1980-12-12,0.513393,0.515625,0.513393,0.513393,0.410525,117258400.0
1980-12-15,0.488839,0.488839,0.486607,0.486607,0.389106,43971200.0


In [5]:
#결측치 조회 (NaN, 음의 무한대, 양의 무한대)
df[df.isin([np.nan, np.inf, -np.inf]).any(1)]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1981-08-10,NaN,NaN,NaN,NaN,NaN,NaN
1981-08-21,0.363839,0.363839,0.359375,0.359375,0.287367,NaN


In [10]:
# 결측치를 제외한 데이터를 출력
# df.dropna(axis=0)
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]

In [ ]:
## 종가만 가지고 테스트
## 시간과 종가만 있는 데이터프레임 
price_df = df.loc[:, ['Adj Close']]
price_df

In [ ]:
price_df.plot()

In [16]:
## 일별 수익율 파생변수 생성 ( 오늘날짜의 종가 - 어제 날짜의 종가) / 어제 날짜의 종가
price_df["daily_rtn"] = price_df["Adj Close"].pct_change()
#price_df["Adj Close"].diff()/price_df["Adj Close"].shift()
# price_df.head()

,Adj Close,daily_rtn
Date,,
1980-12-12,0.410525,NaN
1980-12-15,0.389106,-0.052175
1980-12-16,0.360548,-0.073394
1980-12-17,0.369472,0.024751
1980-12-18,0.380182,0.028987


In [ ]:
## 일별 수익율의 누적곱 파생변수 생성
price_df["st_rtn"] = (1 + price_df["daily_rtn"]).cumprod()
price_df.head()

In [ ]:
price_df["st_rtn"].plot()

In [19]:
first_date = price_df.index[0]
last_date = price_df.index[-1]
print(first_date, last_date)

1980-12-12 2019-06-24


In [20]:
print(price_df.loc[last_date, 'Adj Close'] / price_df.loc[first_date, 'Adj Close'])

485.15924243346933


In [21]:
print(price_df.loc[last_date, 'st_rtn'])

485.15924243347024


In [ ]:
# backtesting 
# cummax() - 종가에서 최고가 저장
# cummin() - 최대 하락 계산
historical_max = price_df["Adj Close"].cummax()
daily_drawdown = price_df["Adj Close"] / historical_max - 1.0
historical_min = daily_drawdown.cummin()
historical_min.plot()

In [27]:
## CAGR : 일별 date 사용 -> 
# 전체 연도를 계산하는 경우 전체 데이터의 기간을 252 영업일을 기준으로 
# 나눈 역수를 제곱한 후 -1
## MDD : 최대 낙폭
## VOL : 연 주가 수익률 표준 편차 = 일별 주가 수익률 * 252일 제곱근
## sharpe : 총 위험 1단위당 초과 수익률
CAGR = price_df.loc[last_date, 'st_rtn'] ** (252/len(price_df))-1
sharpe = np.mean(price_df["daily_rtn"])/np.std(price_df["daily_rtn"])*np.sqrt(252)
VOL = np.std(price_df["daily_rtn"]) * np.sqrt(252)
MDD = historical_min.min()

In [28]:
print(VOL)
print('CAGR : ', round(CAGR*100, 2), '%')
print('Sharpe : ', round(sharpe, 2))
print("VOL : ", round(VOL*100, 2), '%')
print("MDD : ", round(-1*MDD*100, 2), '%')

0.4570483127920232
CAGR :  17.4 %
Sharpe :  0.58
VOL :  45.7 %
MDD :  81.8 %
